<a href="https://colab.research.google.com/github/marceperez99/tesis_analisis_de_datos/blob/main/Analisis_Datos_regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from functools import reduce
import statsmodels.api as sm

# Configuraciones

In [12]:
## Setting ####
# Configuraciones del dataset a usar
# Fecha de inicio de conteo de casos acumulados
CUM_DATE_0 = "2023-07-29"
# Fecha de fin de conteo de casos acumulados
CUM_DATE_F = "2024-01-10"
# Clasificación de casos: SOSPECHOSO, PROBABLE, CONFIRMADO, TOTAL (S+P+C)
CLASSIFICATION_0 =  "CONFIRMADO"
# Nivel: National, Departamento, Eje
LEVEL_0  =  None #"National"
# Nombre de la localidad
NAME_0  = "ALTO PARAGUAY"
# Enfermedad: DENGUE, CHIKUNGUNYA
DISEASE_0 =  "CHIKUNGUNYA"


# argumentos para rollmean media movil
ROLL_MEAN = 3 # periodo
MEAN_ALIGN =  "right" # Alineación de medias calculadas
FILL_MEAN = None # Valor de relleno (Cómo alternativa se puede implementar un metodo de interpolación)

# Semanas de entrenamiento
TRAINING_WINDOW_DAYS = 28
FORECASTING_WINDOW_DAYS = 21

In [3]:
# Read the CSV file
data_csv = pd.read_csv(
    "case_data_2019_2023_upto16sep_23_weekly_v20231103.csv",
    header=0,
    sep=",",
    decimal=".",
    encoding="UTF-8"
)
print(data_csv['level'].unique())

['Department' 'Eje' 'National']


# Preparacion de datos

In [13]:
# Filter rows based on conditions
df_0 = data_csv[(data_csv['disease'] != "DENGUE,CHIKUNGUNYA") & (data_csv['disease'] != "ARBOVIROSIS")]

if DISEASE_0:
  df_0 = df_0[df_0['disease'] == DISEASE_0]
if LEVEL_0:
  df_0 = df_0[df_0['level'] == LEVEL_0]
if NAME_0:
  df_0 = df_0[df_0['name'] == NAME_0]
if CLASSIFICATION_0:
  df_0 = df_0[df_0['classification'] == CLASSIFICATION_0]

# Update the 'i_cases' column in the DataFrame 'df_0' where the value is 0 with a random exponential value between 1e-9 and 1.
df_0.loc[df_0['i_cases'] == 0, 'i_cases'] = np.exp(np.random.uniform(np.log(1e-9), np.log(1)))

# Group by specified columns and apply mutations
df_0 = df_0.groupby(['name', 'level', 'disease', 'classification'],group_keys=False).apply(lambda group: (
    group.assign(csum=(group['i_cases'].cumsum() + np.exp(np.random.uniform(np.log(1e-9), np.log(1)))),
                 id_proy=group['name'] + '-' + group['disease'] + '-' + group['classification'])
)).reset_index(drop=True)
# create columns of time
df_0['date'] = pd.to_datetime(df_0['date'])
df_0['t'] = (df_0['date'] - df_0['date'].min() + (1 * np.timedelta64(1, 'D')))  / np.timedelta64(1, 'D')

print(df_0['id_proy'].unique())

['ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO']


In [5]:
def predict(model, new_data, x, y, confidence_level=0.90):
    # Predicting the mean response
    predicted_values = model.predict(new_data)

    y_pred = model.predict(x)


    squared_errors = (y - y_pred) ** 2
    # Calculate mean squared error
    mse = np.mean(squared_errors)

    sep = np.sqrt(mse / (y_pred.shape[0] - x.shape[1] - 1))
    # TODO: Verificar que se calcule asi el intervalo de confianza
    # Organizing the results into a DataFrame
    return pd.DataFrame({'fit': predicted_values, 'upr': predicted_values + confidence_level*sep, 'lwr': predicted_values - confidence_level*sep})



In [33]:
# Para cada nivel de datos que tenemos
for level in df_0['id_proy'].unique():
  # dataframe base
  df_1 = df_0[df_0['id_proy'] == level]

  for i in range(5, df_1.shape[0] + 1):

    training_dataset = df_1.iloc[i-5:i]
    print(training_dataset.head())

    # Linear model 1) lm.e.tc: I(log(csum)) ~ t
    t = training_dataset['t']
    x = t.values.reshape(-1, 1)
    y = np.log(training_dataset['csum'])

    lm_e_tc = LinearRegression().fit(x, y)

    # Linear model 2) lm.se.tc: I(log(csum)) ~ log(t)
    lm_se_tc = LinearRegression().fit(np.log(t.values).reshape(-1, 1), np.log(training_dataset['csum']))


    # Linear model 3) lm.seco.tc: I(log(csum)) ~ log(t) + t
    lm_seco_tc = LinearRegression().fit(np.column_stack((np.log(t.values), t.values)), np.log(training_dataset['csum']))


    results = pd.DataFrame({
      'Model': ['log(TC)=log(a)+c t', 'log(TC)=log(a)+b log(t)', 'log(TC)=log(a)+b log(t)+c t'],
      'Method': ['Exponential', 'Sub-exponential', 'Sub-exponential-decay'],
      'log_a': [lm_se_tc.intercept_, lm_se_tc.intercept_, lm_seco_tc.intercept_],
      'b': [lm_se_tc.coef_[0] if hasattr(lm_se_tc, 'coef_') else 0,
            lm_se_tc.coef_[0] if hasattr(lm_se_tc, 'coef_') else 0,
            lm_seco_tc.coef_[0] if hasattr(lm_seco_tc, 'coef_') else 0],
      'c': [lm_se_tc.coef_[1] if hasattr(lm_e_tc, 'coef_') and len(lm_se_tc.coef_) > 1 else 0,
            0,
            lm_seco_tc.coef_[1] if hasattr(lm_seco_tc, 'coef_') and len(lm_seco_tc.coef_) > 1 else 0],
      'Fit.criteria': [np.sum((lm_se_tc.predict(np.log(t.values).reshape(-1, 1)) - y) ** 2),
                      np.sum((lm_se_tc.predict(np.log(t.values).reshape(-1, 1)) - y) ** 2),
                      np.sum((lm_seco_tc.predict(np.column_stack((np.log(t.values), t.values))) - y) ** 2)],
      'id': np.full(3, i)
    })
    # TODO: PARAMETRIZAR SEMANAS A PREDECIR
    new = pd.DataFrame({'t': t.max(skipna=True) + np.array([0, 7, 14, 21, 28])})
    new['Method']= np.full(5, 'Exponential')
    data = predict(lm_e_tc, new['t'].values.reshape(-1,1),x,y)
    data['t'] = new['t']
    data['Method'] = new['Method']
    exponential = pd.merge(results, data, on='Method',how='right')



    new['Method']= np.full(5, 'Sub-exponential')
    data = predict(lm_se_tc, new['t'].values.reshape(-1,1),x,y)
    data['t'] = new['t']
    data['Method'] = new['Method']
    sub_exponential = pd.merge(results, data, on='Method',how='right')

    new['Method']= np.full(5, 'Sub-exponential-decay')
    data = predict(lm_seco_tc, new['t'].values.reshape(-1,1),x,y)
    data['t'] = new['t']
    data['Method'] = new['Method']
    sub_exponential_decay = pd.merge(results, data, on='Method',how='right')

    result = pd.concat([exponential, sub_exponential, sub_exponential_decay], ignore_index=True)
    print(result)
    input()





        date           name       level      disease classification  i_cases  \
0 2023-01-07  ALTO PARAGUAY  Department  CHIKUNGUNYA     CONFIRMADO      1.0   
1 2023-01-14  ALTO PARAGUAY  Department  CHIKUNGUNYA     CONFIRMADO      2.0   
2 2023-01-21  ALTO PARAGUAY  Department  CHIKUNGUNYA     CONFIRMADO      1.0   
3 2023-01-28  ALTO PARAGUAY  Department  CHIKUNGUNYA     CONFIRMADO      3.0   
4 2023-02-04  ALTO PARAGUAY  Department  CHIKUNGUNYA     CONFIRMADO      6.0   

     Population  incidence  csum                               id_proy     t  
0  19298.158536   5.181842   1.0  ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO   1.0  
1  19298.158536  10.363683   3.0  ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO   8.0  
2  19298.158536   5.181842   4.0  ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO  15.0  
3  19298.158536  15.545525   7.0  ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO  22.0  
4  19298.158536  31.091049  13.0  ALTO PARAGUAY-CHIKUNGUNYA-CONFIRMADO  29.0  


ValueError: X has 1 features, but LinearRegression is expecting 2 features as input.

In [ ]:
max_proy_i = df_0['proy_i'].max(skipna=True)

# Iterate over the range from 1 to max_proy_i
for proy in range(1,max_proy_i + 1):
    df = df_0[df_0['proy_i'] == proy]
    max_id = df['id'].max(skipna=True)
    # Iterate over the range from 5 to max_id
    for var in range(5, max_id + 1):
      df1 = df[df['id'] <= var]

      d_t = 28
      # Find the maximum date in df1
      max_date = df1['date'].max()

      # Calculate start_date and end_date
      start_date = max_date - timedelta(days=d_t)
      end_date = max_date

      # Subset df1 based on date conditions
      d = df1[(df1['date'] >= start_date) & (df1['date'] <= end_date)].copy()
      print(d.size)
      # Convert 'date' column to datetime
      d['date'] = pd.to_datetime(d['date'])

      t = (d['date'] - d['date'].min() + (1 * np.timedelta64(1, 'D')))  / np.timedelta64(1, 'D')
      d['t'] = t
      # Linear model lm.e.tc: I(log(csum)) ~ t
      x = t.values.reshape(-1, 1)
      y = np.log(d['csum'])
      lm_e_tc = LinearRegression().fit(x, y)


      # Linear model lm.se.tc: I(log(csum)) ~ log(t)

      lm_se_tc = LinearRegression().fit(np.log(t.values).reshape(-1, 1), np.log(d['csum']))


      # Linear model lm.seco.tc: I(log(csum)) ~ log(t) + t
      lm_seco_tc = LinearRegression().fit(np.column_stack((np.log(t.values), t.values)), np.log(d['csum']))


      # Model names
      modelo = [
          "log(TC)=log(a) + ct",
          "log(TC)=log(a) + blog(t)",
          "log(TC)=log(a) + blog(t) + ct"
      ]
      # Method names
      metodo = {
          "log(TC)=log(a) + ct": "Exponential",
          "log(TC)=log(a) + blog(t)": "Sub-exponential",
          "log(TC)=log(a) + blog(t) + ct": "Sub-exponential-decay"
      }

      # # Coefficients for parameter 'a'
      par_a = {
          "log(TC)=log(a) + ct`": np.exp(lm_e_tc.intercept_),
          "log(TC)=log(a) + blog(t)": np.exp(lm_se_tc.intercept_),
          "log(TC)=log(a) + blog(t) + ct": np.exp(lm_seco_tc.intercept_)
      }

      # # Coefficients for parameter 'b'
      par_b = {
          "log(TC)=log(a) + ct": 0,
          "log(TC)=log(a) + blog(t)": lm_se_tc.coef_[0],
          "log(TC)=log(a) + blog(t) + ct": lm_seco_tc.coef_[0]
      }

      par_c = {
          "log(TC)=log(a) + ct": lm_e_tc.coef_[0],
          "log(TC)=log(a) + blog(t)": 0,
          "log(TC)=log(a) + blog(t) + ct": lm_seco_tc.coef_[1]
      }

      # # Compare models

      model_ctest = pd.DataFrame({
          "Model": modelo,
          "log(a)": par_a.values(),
          "b": par_b.values(),
          "c": par_c.values(),
          "Metodo": [metodo[model] for model in modelo]
      })

      # # 4-week projections
      new_t = max(t, default=0) + np.array([0, 7, 14, 21, 28])
      new_y =  d.loc[d['t'].isin(new_t), 'csum'].to_numpy()
      # print(max(t, default=0))





60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
60
6

KeyboardInterrupt: 